<a href="https://colab.research.google.com/github/RafaelBi/machine_learning/blob/Playground/InceptionResNetV2_plates_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pretrained ResNetv2, 
40 pics on train, 
Kaggle cleaned vs dirty v2 score 0.895, https://www.kaggle.com/c/platesv2


In [0]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, Activation, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import applications, optimizers
print(os.listdir("drive/My Drive/Plates"))



In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

In [0]:
img_size = 224
batch_size = 20

In [0]:
%%time
with tf.device('/device:GPU:0'):
  train_datagen=ImageDataGenerator(
      rescale=1.0/255.0,       
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip = True
      )

  train_generator = train_datagen.flow_from_directory(
          'drive/My Drive/Plates/train',
          target_size=(img_size, img_size),
          batch_size=batch_size,
          class_mode='binary')

  test_datagen = ImageDataGenerator(rescale=1.0/255.0)
  test_generator = test_datagen.flow_from_directory(  
          'drive/My Drive/Plates',
          classes=['test'],
          target_size = (img_size, img_size),
          batch_size = 1,
          shuffle = False,        
          class_mode = None)


In [0]:
with tf.device('/device:GPU:0'):
  base_model = applications.InceptionResNetV2(weights='imagenet', 
                          include_top=False, 
                          input_shape=(img_size, img_size, 3))

In [0]:
base_model.trainable = False

In [0]:
base_model.summary()

In [0]:
with tf.device('/device:GPU:0'):
  add_model = Sequential()
  add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
  add_model.add(Dense(1024, activation='relu'))
  add_model.add(Dropout(0.5))
  add_model.add(Dense(1, activation='sigmoid'))

  model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
  model.compile(loss='binary_crossentropy', optimizer = optimizers.rmsprop(lr=0.0001, decay=1e-6), metrics=['accuracy'])

model.summary()
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [0]:
#another way to add layer
with tf.device('/device:GPU:0'):
  x = base_model.output
  x = Flatten()(x)
  x = Dense(1024, activation="relu")(x)
  x = Dropout(0.5)(x)
  predictions = Dense(1, activation="sigmoid")(x)
  model = Model(input = base_model.input, output = predictions)

model.summary()

In [0]:
%%time
with tf.device('/device:GPU:0'):
  history=[]
  model.compile(loss='binary_crossentropy', optimizer = optimizers.rmsprop(lr=0.0001, decay=1e-6), metrics=['accuracy'])
  history=model.fit_generator(
      train_generator,
      steps_per_epoch=200,
      epochs=20,
      verbose=0)
  #500 steps, 25 epoch original

output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [0]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='accuracy')
#plt.plot(history.history['loss'], label='loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

In [0]:
plt.plot(history.history['loss'], label='loss')
plt.xlabel('Epoch')
plt.ylabel('loss-value')

In [0]:
%time
with tf.device('/device:GPU:0'):
  test_generator.reset()
  predict = model.predict_generator(test_generator, steps = len(test_generator.filenames))
  len(predict)
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [0]:
sub_df = pd.read_csv('drive/My Drive/Plates/sample_submission.csv')
sub_df.head()

In [0]:
sub_df['label'] = predict
sub_df['label'] = sub_df['label'].apply(lambda x: 'dirty' if x > 0.5 else 'cleaned')
sub_df.head()

In [0]:
sub_df['label'].value_counts()

In [0]:
sub_df.to_csv('Res_net_200steps_20epochs.csv', index=False)